In [1]:
import glob
import pandas as pd
import xml.etree.ElementTree as ET
from datetime import datetime

In [2]:
def extract_from_csv(file):
    dataframe = pd.read_csv(file)
    return dataframe

In [3]:
def extract_from_json(file):
    dataframe = pd.read_json(file, lines=True)
    return dataframe

In [4]:
def extract_from_xml(file_to_process):
    dataframe = pd.DataFrame(columns=['car_model','year_of_manufacture','price','fuel'])
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    cars = root.findall(".//row")
    for row in cars:
        car_model = row.find("car_model").text
        year_of_manufacture = row.find("year_of_manufacture").text
        price = float(row.find("price").text)
        fuel = row.find("fuel").text
        dataframe = dataframe.append({"car_model":car_model,"year_of_manufacture":year_of_manufacture,"price":price,"fuel":fuel},
                                     ignore_index=True)
    return dataframe

In [5]:
def extract():
    extracted_data = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel']) 
 
    for csvfile in glob.glob("*.csv"):
        extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
    
    for jsonfile in glob.glob("*.json"):
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
 
    for xmlfile in glob.glob("*.xml"):
        extracted_data = extracted_data.append(extract_from_xml(xmlfile), ignore_index=True)
        
    return extracted_data

In [6]:
def transform(data):
        data['year_of_manufacture'] = pd.to_datetime(data['year_of_manufacture'], format='%Y', errors='ignore').dt.year
        return data

In [7]:
def load(targetfile,data_to_load):
    data_to_load.to_csv(targetfile, columns=['car_model','year_of_manufacture','price','fuel'], index=False)  

In [8]:
def log(message):
    now = datetime.now() 
    timestamp = now.strftime('%Y-%h-%d-%H:%M:%S')
    with open("logfile.txt","a") as f: 
        f.write(timestamp + ',' + message + '\n')

In [9]:
log("ETL Job Started")
log("Extract phase Started")
extracted_data = extract()
log("Extract phase Ended")

In [10]:
log("Transform phase Started")
transformed_data = transform(extracted_data)
log("Transform phase Ended")

In [11]:
log("Load phase Started")
load("transformed_data.csv",transformed_data)
log("Load phase Ended")
log("ETL Job Ended")
transformed_data

,car_model,year_of_manufacture,price,fuel
0,ritz,2014,5000.000000,Petrol
1,sx4,2013,7089.552239,Diesel
2,ciaz,2017,10820.895522,Petrol
3,wagon r,2011,4253.731343,Petrol
4,swift,2014,6865.671642,Diesel
...,...,...,...,...
85,camry,2006,3731.343284,Petrol
86,land cruiser,2010,52238.805970,Diesel
87,corolla altis,2012,8805.970149,Petrol
88,etios liva,2013,5149.253731,Petrol
